<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Tue Jun 30 23:32:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [5]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-30T23-33-13'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 113 (delta 1), reused 65 (delta 1), pack-reused 0
Receiving objects: 100% (113/113), 985.08 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-30T23-33-13
total 69
drwx------  4 root root  4096 Jun 30 23:33 data
-rw-------  1 root root 35149 Jun 30 23:33 LICENSE
drwx------  2 root root  4096 Jun 30 23:33 notebooks
-rw-------  1 root root 10499 Jun 30 23:33 README.md
drwx------  3 root root  4096 Jun 30 23:33 reports
drwx------  2 root root  4096 Jun 30 23:33 requirements
drwx------ 13 root root  4096 Jun 30 23:33 src
drwx------  3 root root  4096 Jun 30 23:33 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 1.4MB 8.4MB/s 
     |████████████████████████████████| 4.3MB 16.1MB/s 
     |████████████████████████████████| 225kB 44.1MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 460kB 44.7MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 112kB 53.6MB/s 
     |████████████████████████████████| 112kB 46.3MB/s 
     |████████████████████████████████| 133kB 53.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for pathos: filename=pathos-0.2.6-cp36-none-any.whl size=77673 sha256=6876236bebcf715e9ea7db2851ff267cc6bff62f7b126bb703cf013c23a24ba5
  Stored in directory: /root/.cache/pip/wheels/3a/e8/c8/04cdd0c4bc6fbce35f642fc004244228916daae74bb0f482da
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-30T23-33-13/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/processed/seq2seq/',
  'max_input_length': 128,
  'max_output_length': 8,
  'input_vocab_size': 5000,
  'input_embedding_dim': 50,
  'output_vocab_size': 5000,
  'output_embedding_dim': 50,
  'latent_dim': 320,
  'learning_rate': 0.001,
  'dropout_rate': 0.05,
  'epochs': 50,
  'early_stopping_patience': 3,
  'early_stopping_min_delta': 0.001,
  'evaluation_dataset': 'validation',
  'batch_size': 1024,
  'random_seed': 1,
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=1024, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', dropout_rate=0.05, early_stopping_min_delta=0.001, early_stopping_patience=3, epochs=50, evaluation_dataset='validation', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=5000, latent_dim=320, learning_rate=0.001, max_input_length=128, max_output_length=8, output_embedding_dim=50, output_vocab_size=5000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  250000    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 724,880
Trainable param

wandb: Visualizing heatmap.


epoch 1 evaluation on test data time: 20.025397777557373 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.753346>, 'test_rouge_1_p': 0.239067573051948, 'test_rouge_1_r': 0.10253368748067401, 'test_rouge_1_f1': 0.1381500672763612, 'test_rouge_2_p': 0.005516943993506494, 'test_rouge_2_r': 0.0017303334686147183, 'test_rouge_2_f1': 0.0024998067408781694, 'test_rouge_3_p': 0.0, 'test_rouge_3_r': 0.0, 'test_rouge_3_f1': 0.0, 'test_rouge_L_p': 0.23904854910714282, 'test_rouge_L_r': 0.10252544377125844, 'test_rouge_L_f1': 0.13813859251600252}


...

epoch 50 - batch 10 - loss 0.950724720954895
epoch 50 - batch 20 - loss 0.9580774307250977
epoch 50 - batch 30 - loss 0.9733214378356934
epoch 50 - batch 40 - loss 0.9872172474861145
epoch 50 - batch 50 - loss 0.9304069876670837
epoch 50 - batch 60 - loss 0.951403796672821
epoch 50 - batch 70 - loss 1.0181962251663208
epoch 50 - batch 80 - loss 0.9546391367912292
epoch 50 - batch 90 - loss 0.9173582196235657
epoch 50 - batch 100 - loss 0.9559993743896484
epoch 50 - batch 110 - loss 0.9671520590782166
epoch 50 - batch 120 - loss 0.9621896147727966
epoch 50 - batch 130 - loss 0.901062548160553
epoch 50 - batch 140 - loss 0.9035749435424805
epoch 50 - batch 150 - loss 0.947558581829071
epoch 50 - batch 160 - loss 0.9556016325950623
epoch 50 - batch 170 - loss 0.9278303980827332
epoch 50 - batch 180 - loss 0.9166904091835022
epoch 50 - batch 190 - loss 0.9502372741699219
epoch 50 - batch 200 - loss 0.9440442323684692
epoch 50 - batch 210 - loss 0.9642869234085083
epoch 50 - batch 220 - los

wandb: Visualizing heatmap.


epoch 50 evaluation on test data time: 22.752963304519653 sec
epoch evaluation:  {'epoch': 50, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.265295>, 'test_rouge_1_p': 0.5710893714730211, 'test_rouge_1_r': 0.5373749251120902, 'test_rouge_1_f1': 0.5448867129753139, 'test_rouge_2_p': 0.30549390388257586, 'test_rouge_2_r': 0.29448453564664495, 'test_rouge_2_f1': 0.29676273493428024, 'test_rouge_3_p': 0.13609814664502168, 'test_rouge_3_r': 0.13055541463744588, 'test_rouge_3_f1': 0.1317300174094259, 'test_rouge_L_p': 0.5691989952941404, 'test_rouge_L_r': 0.5358522546092299, 'test_rouge_L_f1': 0.5432466466360792}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


Saving checkpoint...
Saved checkpoint: models/checkpoints/baseline/ckpt-18
Done saving model
